# Asynchronous Programming
- 아래의 용어들과 혼용되지만, 주 실행흐름을 멈추지 않는 프로그래밍 방식을 말한다.

- Synchronous  : 통지 모델. application이 kernel을 체크한다.
- Asynchronous : 통지 모델. kernel이 application에게 결과를 통지한다.
- Blocking     : application이 Kernel의 응답을 대기한다. (결과가 나왔을 때 return)
- Not-blocking : application이 kernel의 응답을 대기하지 않는다. (결과에 상관 없이 return)

- Concurrency  : 실행 순서와 무관하게 동작
- Parallelism  : 많은 작업을 물리적으로 동시에 수행.

## 기본 라이브러리

In [1]:
const _ = require('./underscore-min.js');

1.0 학생, 과목 정보 확인
  - const, let
  - scope
  - sync / async

In [ ]:
{
    const fs = require('fs');
    const student = fs.readFileSync('./data/학생.csv', {encoding:'UTF-8'});
    const subject = fs.readFileSync('./data/교과목.csv', {encoding:'UTF-8'});
    console.log(student, subject);
}

1.1 중복제거
  - 함수만들기

In [ ]:
{
    const readFile = filename => {
        const fs = require('fs');
        return fs.readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    };
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

1.2 좀 더 줄여보기
  - 화살표 함수의 생략

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

2.0 JSON으로 파싱하기
  - 함수 반환 하기
  - _.map
  - _.reduce

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const body    = csv.split('\n');
        const header  = body.shift().split(',');
        return _.map(body, data=>{
            return _.reduce(data.split(','), (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.1 조금 줄여 보기
  - 배열 비구분해 할당

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body] = _.map(csv.split('\n'), d=>d.split(','));
        return _.map(body, data=>{
            return _.reduce(data, (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.2 COMPOSE 사용해 보기
  - 함수로 분해
  - _.compose

In [ ]:
{
    const readFile  = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const splitCr   = text => text.split('\n');
    const splitComma= text => text.split(',');
    const makeFunctionObjWithHeader = header => array => _.reduce(array, (m, v, i)=>(m[header[i]]=v,m),{});
    const csvToObj = csv => {
        const [header, ...body] = splitCr(csv);
        const funcArrayToObj = _.compose(
            _.compose(makeFunctionObjWithHeader, splitComma)(header),
            splitComma
        );
        return _.map(body, funcArrayToObj);
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

2.3 줄여보기

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

3.0 시험 데이터 만들기
  - for ... of

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const makeScore = () => Math.round(Math.random() * 41 + 60);
    
    const student = csvToObj(readFile('학생.csv'));
    const subject = csvToObj(readFile('교과목.csv'));
    
    for(const i of student) {
        for(const j of subject) {
            console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
        }
    }
}

3.1 다른 순환 방법
  - _.each

In [9]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const makeScore = () => Math.round(Math.random() * 41 + 60);
    
    const student = csvToObj(readFile('학생.csv'));
    const subject = csvToObj(readFile('교과목.csv'));
    
    _.each(student, i => {
       _.each(subject, j => {
           console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
       });
    });
    console.log('end');
}

100,100,97
100,200,66
100,300,81
100,400,99
100,500,93
100,600,69
110,100,91
110,200,68
110,300,85
110,400,88
110,500,78
110,600,98
120,100,70
120,200,77
120,300,79
120,400,90
120,500,86
120,600,89
130,100,77
130,200,81
130,300,85
130,400,94
130,500,79
130,600,67
140,100,65
140,200,78
140,300,74
140,400,93
140,500,84
140,600,74
150,100,93
150,200,65
150,300,91
150,400,77
150,500,61
150,600,85
160,100,71
160,200,80
160,300,68
160,400,69
160,500,87
160,600,71
end


4.0 시험 더미 파일 생성